사람 + 총 + 칼 인식
pose estimation은 모든 사람에 대해 인식(수정해서 아래 경우에만 적용할 수도 있음)
사람과 총 혹은 칼의 bounding box가 겹치는 경우에만 arcface 적용

In [ ]:
!pip install opencv-python


In [1]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import cv2
import mediapipe as mp
import time
import torch
import numpy as np

from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from insightface.app import FaceAnalysis

#######################
# 1) YOLO + ArcFace 초기화
#######################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLOv8 모델 로드
model = YOLO("C:/epoch180.pt").to(device)

# ArcFace 초기화
arc_app = FaceAnalysis(name="buffalo_l")
arc_app.prepare(ctx_id=-1, det_size=(640,640))  # CPU 사용 예시

# 내 얼굴 임베딩
my_face_embedding = np.load("my_face_embedding.npy")

def get_face_embedding(arc_app, face_img_bgr):
    faces = arc_app.get(face_img_bgr)
    if len(faces) == 0:
        return None
    return faces[0].embedding

def is_my_face(face_embedding, my_embedding, threshold=0.4):
    sim = cosine_similarity([face_embedding], [my_embedding])[0][0]
    return (sim > threshold), sim

#######################
# 2) Mediapipe Pose 초기화
#######################
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 (Mediapipe Pose)
LEFT_SHOULDER = 11
RIGHT_SHOULDER = 12
LEFT_WRIST = 15
RIGHT_WRIST = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    # Mediapipe Pose: y=0 상단, y=1 하단 (작을수록 위)
    return wrist_y < (shoulder_y - threshold)

#######################
# 3) 바운딩박스 overlap(교차) 함수
#######################
def boxes_overlap(boxA, boxB):
    """
    boxA, boxB = (x1, y1, x2, y2)
    단순 교차 여부 판단 (한 픽셀이라도 겹치면 True)
    """
    x1A, y1A, x2A, y2A = boxA
    x1B, y1B, x2B, y2B = boxB

    # 수평 방향 교차 판단
    overlap_x = not (x2A < x1B or x2B < x1A)
    # 수직 방향 교차 판단
    overlap_y = not (y2A < y1B or y2B < y1A)

    return overlap_x and overlap_y

#######################
# 4) 메인 루프
#######################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽어올 수 없습니다.")
        break

    # Pose(전신) 추론 (원본 프레임 전체)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pose_results = pose.process(rgb)

    # 간단 팔 상태 판별
    action_text = ""
    if pose_results.pose_landmarks:
        landmarks = pose_results.pose_landmarks.landmark
        left_shoulder_y = landmarks[LEFT_SHOULDER].y
        right_shoulder_y = landmarks[RIGHT_SHOULDER].y
        left_wrist_y = landmarks[LEFT_WRIST].y
        right_wrist_y = landmarks[RIGHT_WRIST].y

        left_arm_up = is_arm_raised(left_shoulder_y, left_wrist_y, threshold=0.05)
        right_arm_up = is_arm_raised(right_shoulder_y, right_wrist_y, threshold=0.05)

        if left_arm_up and right_arm_up:
            action_text = "both arms up"
        elif left_arm_up:
            action_text = "left arm up"
        elif right_arm_up:
            action_text = "right arm up"
        else:
            action_text = "do nothing"

        # 전신 랜드마크 시각화
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # 왼쪽 상단에 pose 텍스트 표시 (전체)
    cv2.putText(frame, action_text, (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # ---------------------
    # YOLO 추론
    # ---------------------
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(rgb_frame)

    # (1) 박스 분류: 사람 vs 무기(총=1, 칼=2)
    person_boxes = []
    weapon_boxes = []  # 총(1), 칼(2)

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)
        conf = float(box.conf)
        class_name = model.names[class_id]

        # 사람/총/칼 시각화
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        label = f"{class_name}: {conf:.2f}"
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

        if class_id == 0: # person
            person_boxes.append((x1, y1, x2, y2))
        elif class_id in [1, 2]: # gun or knife
            weapon_boxes.append((x1, y1, x2, y2))

    # (2) 무기와 겹치는 사람만 Arcface 시도
    for (px1, py1, px2, py2) in person_boxes:
        # 무기와 교차?
        overlap_with_weapon = False
        for wbox in weapon_boxes:
            if boxes_overlap((px1, py1, px2, py2), wbox):
                overlap_with_weapon = True
                break

        if overlap_with_weapon:
            # --------------- ArcFace 처리 ---------------
            person_crop = frame[py1:py2, px1:px2]  # BGR
            if person_crop.size == 0:
                continue

            face_embedding = get_face_embedding(arc_app, person_crop)
            if face_embedding is not None:
                same_person, sim = is_my_face(face_embedding, my_face_embedding, threshold=0.4)
                if same_person:
                    label = f"Me! (sim={sim:.2f})"
                    color = (0, 255, 0)
                else:
                    label = f"Not me (sim={sim:.2f})"
                    color = (0, 0, 255)

                cv2.rectangle(frame, (px1, py1), (px2, py2), color, 2)
                cv2.putText(frame, label, (px1, py1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time
    cv2.putText(frame, f"FPS: {fps:.2f}", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("YOLO_Arcface_MediaPipe_Weapons", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()